<a href="https://colab.research.google.com/github/BritCrit/GTM-Container-Checker/blob/main/Python3_GTM_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Please upload your site list CSV with no headers (check's Google Cache and site directly)
!pip install requests-html lxml_html_clean  # Install the necessary package
from requests_html import HTMLSession # Import HTMLSession
import re
import pandas as pd

from datetime import date
!rm -rf sample_data
print('Please upload csv of sites you would like to check')
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
reportname =  fn
GTM_Code = "GTM-NV92H48" #@param ["GTM-NV92H48"] {allow-input: true}
google_cache = 'http://webcache.googleusercontent.com/search?q=cache:'


def test_site(url):
  confirmed = False
  correct_url = False
  codes = []
  gtm_code_list = []
  try:
    session = HTMLSession()
    r = session.get(google_cache+url+'&strip=0&vwsrc=1')
    correct_url = True
  except:
    print(f"Invalid URL: {url}")
    return "Invalid URL"
  if correct_url:
    GTM_locations = [m.start() for m in re.finditer('GTM', r.text)]
    for location in GTM_locations:
      found_gtm_code = r.text[location:(location+11)]
      gtm_code_list.append(found_gtm_code)
      gtm_code_list = list(set(gtm_code_list))
    print(f"There were {len(gtm_code_list)} GTM tags found for {url}. GTM code(s): {', '.join([str(x) for x in gtm_code_list]) }")
    if GTM_Code in gtm_code_list:
      print(f"Our GTM code was found! on {url}")
      confirmed = True
      codes = ', '.join(gtm_code_list)
    else:
      print(f"Our GTM code was NOT found on {url}.")
      codes = 'No GTM found'
    return codes

def test_site_no_cache(url):
  confirmed = False
  correct_url = False
  codes = []
  gtm_code_list = []
  try:
    session = HTMLSession()
    r = session.get(url)
    correct_url = True
  except:
    print(f"Invalid URL: {url}")
    return "Invalid URL"
  if correct_url:
    GTM_locations = [m.start() for m in re.finditer('GTM', r.text)]
    for location in GTM_locations:
      found_gtm_code = r.text[location:(location+11)]
      gtm_code_list.append(found_gtm_code)
      gtm_code_list = list(set(gtm_code_list))
    print(f"There were {len(gtm_code_list)} GTM tags found for {url}. GTM code(s): {', '.join([str(x) for x in gtm_code_list]) }")
    if GTM_Code in gtm_code_list:
      print(f"Our GTM code was found! on {url}")
      confirmed = True
      codes = ', '.join(gtm_code_list)
    else:
      print(f"Our GTM code was NOT found on {url}.")
      codes = 'No GTM found'
    return codes

def confirm_gtm(df):
  df['verified'] = df.gtm_codes.str.contains(GTM_Code)
  return df


today = date.today().strftime("%m_%d_%y")

df = pd.read_csv(fn,header=None)
df.columns = ['site']
df['gtm_codes'] = df['site'].apply(lambda x: test_site(x))
df = confirm_gtm(df)

cached_df = df.loc[df.gtm_codes != 'No GTM found'].copy()

print("Performing double check against GTM's not found...")
new_df = df.loc[df.gtm_codes == 'No GTM found'].copy()
new_df = new_df.reset_index(drop=True).drop(['gtm_codes', 'verified'], axis=1)
new_df['gtm_codes'] = new_df['site'].apply(lambda x: test_site_no_cache(x))
new_df = confirm_gtm(new_df)

print("Combining final results into single dataframe...")
final_df = pd.concat([cached_df, new_df]).reset_index(drop=True)

final_df.to_csv(f"{fn.split('.')[0]}_gtm_check{today}.csv")

!zip -r /content/GTMcheck.zip /content/
files.download("/content/GTMcheck.zip")

final_df
### OLD TEST CODE ###
# import requests
# from bs4 import BeautifulSoup

# def gtm_return(code):
#   return re.search("GTM-[A-Z0-9]{6,7}",str(code)).group(0)

# def test_site(url):
#   confirmed = False
#   correct_url = False
#   codes = []
#   try:
#     user_agent = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'
#     headers = {'User-Agent' : user_agent}
#     req = requests.get(url, headers)
#     correct_url = True
#   except:
#     print(f"Invalid URL: {url}")
#     return "Invalid URL"
#   if correct_url:
#     soup = BeautifulSoup(req.content, 'html.parser')
#     GTM = soup.head.findAll(text=re.compile(r'GTM'))
#     gtm_code_list = [gtm_return(GTM) for code in GTM]
#     print(f"There were {len(GTM)} GTM tags found for {url}. GTM code(s): {', '.join([str(x) for x in gtm_code_list]) }")
#     if GTM_Code in gtm_code_list:
#       print(f"Our GTM code was found! on {url}")
#       confirmed = True
#       codes = ', '.join(gtm_code_list)
#     else:
#       print(f"Our GTM code was NOT found on {url}.")
#       codes = 'No GTM found'
#     return codes

Please upload csv of sites you would like to check


Saving GTM Check_Northern.csv to GTM Check_Northern (1).csv
User uploaded file "GTM Check_Northern (1).csv" with length 1871 bytes
There were 0 GTM tags found for https://www.arbeatty.com/. GTM code(s): 
Our GTM code was NOT found on https://www.arbeatty.com/.
There were 0 GTM tags found for https://acepowerequipment.net/. GTM code(s): 
Our GTM code was NOT found on https://acepowerequipment.net/.
There were 0 GTM tags found for https://www.ahearnequipment.com/. GTM code(s): 
Our GTM code was NOT found on https://www.ahearnequipment.com/.
There were 0 GTM tags found for https://applefarmservice.com/. GTM code(s): 
Our GTM code was NOT found on https://applefarmservice.com/.
There were 0 GTM tags found for https://www.artmanequipmentinc.com/. GTM code(s): 
Our GTM code was NOT found on https://www.artmanequipmentinc.com/.
There were 0 GTM tags found for https://www.bane-welkerkubota.com/. GTM code(s): 
Our GTM code was NOT found on https://www.bane-welkerkubota.com/.
There were 0 GTM ta

In [1]:
from bs4 import BeautifulSoup
import requests
l={}
u=list()
country_code = '1'
url="https://www.proxynova.com/proxy-server-list/country-us/"
respo = requests.get(url).text
soup = BeautifulSoup(respo,'html.parser')
allproxy = soup.find_all("tr")


In [2]:
allproxy

[<tr>
 <th>Proxy IP</th>
 <th>Proxy Port</th>
 <th>Last Check</th>
 <th nowrap="nowrap"><span title="Proxy Speed in bytes per second">Proxy Speed</span></th>
 <th>
 <abbr title="Percent of times proxy was found to be responsive (total checks) - past 24 hours">Uptime</abbr>
 </th>
 <th><span title="The location of that particular proxy.">Proxy Country</span></th>
 <th>Anonymity</th>
 </tr>,
 <tr data-proxy-id="2022145">
 <td align="left" title="2,022,145">
 <abbr title="ec2-44-226-167-102.us-west-2.compute.amazonaws.com">
 <script>document.write(atob("NDQuMjI2LjE2Ny4x").concat("0220".substring(0+0, 4-2)))</script>
 </abbr>
 </td>
 <td align="left">
 <a href="/proxy-server-list/port-3128/" title="Port 3128 proxies">3128</a>
 </td>
 <td align="left">
 <time class="icon icon-check timeago" datetime="2025-02-04 21:56:21Z"></time>
 </td>
 <td align="left">
 <div class="progress-bar">
 <div class="progress-bar-inner" style="width:15.55%; background-color: hsl(18,100%,50%);"></div>
 </div>
 <s

In [ ]:
for proxy in allproxy:
 foo = proxy.find_all("td")
 try:
   l["ip"]=foo[0].text.replace("\n","").replace("document.write","").replace(")”,").replace("\'","").replace(";","")
 except:
   l["ip"]=None
 try:
  l["port"]=foo[1].text.replace("\n",").replace(" ",")
 except:
  l["port"]=None
 try:
  l["country"]=foo[5].text.replace("\n",").replace(" ",")
 except:
  l["country"]=None
 if(l["port"] is not None):
  u.append(l)

  l={}
print(u)

[{'ip': None, 'port': ').replace(,                                    8080).replace(,                            ', 'country': ').replace(,).replace(,United States).replace(,).replace(,                                             - Clifton ).replace(,).replace(,'}, {'ip': None, 'port': ').replace(,                                    3128).replace(,                            ', 'country': ').replace(,).replace(,United States).replace(,).replace(,                    ).replace(,                ).replace(,'}, {'ip': None, 'port': ').replace(,                                    8080).replace(,                            ', 'country': ').replace(,).replace(,United States).replace(,).replace(,                                             - Paramount ).replace(,).replace(,'}, {'ip': None, 'port': ').replace(,                                    8080).replace(,                            ', 'country': ').replace(,).replace(,United States).replace(,).replace(,                                     

In [ ]:
for proxy in allproxy:
  foo = proxy.find_all("td")
  try:
    l['ip']=foo[0].text.rep

[]
[<td align="left">
<abbr>
<script>document.write('64.22' + '5.16.208');</script>
</abbr>
</td>, <td align="left">
                                    8080
                            </td>, <td align="left">
<time class="icon icon-check timeago" datetime="2021-11-17 20:51:14Z"></time>
</td>, <td align="left">
<div class="progress-bar" data-value="35.045175" title="1601"></div>
<small>1601 ms</small>
</td>, <td class="text-center text-sm">
<span class="uptime-low">29%</span>
<span> (581)</span>
</td>, <td align="left">
<img alt="us" class="flag flag-us inline-block align-middle" src="/assets/images/blank.gif"/>
<a href="/proxy-server-list/country-us/" title="Proxies from United States">United States

                                            <span class="proxy-city"> - Clifton </span>
</a>
</td>, <td align="left">
<span class="proxy_elite font-weight-bold smallish">Elite</span>
</td>]
[<td align="left">
<abbr>
<script>document.write('20.81' + '.62.32');</script>
</abbr>
</td>, <td 